<a href="https://colab.research.google.com/github/Resmi-p/Analysing-the-popularity-of-films-and-TV-shows-using-pyspark-and-spark-sql/blob/main/PySpark_Assignment_Sep23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Mounted at /content/drive


In [ ]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [ ]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/MyDrive/ADS2/Assignment /all-weeks-countries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [ ]:
# CodeGrade Tag Ex1a

### Display the table and its schema
netflixDF.show()
netflixDF.printSchema()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                null|                         1|
|   Argentina|          AR|2022-08

In [ ]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF
filmsDF = netflixDF.filter("category == 'Films'")
tvDF = netflixDF.filter("category == 'TV'")

In [ ]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table
filmsDF = filmsDF.drop('season_title')
filmsDF.show()


+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [ ]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.
season_title_isnull = F.when(F.isnull('season_title'), F.col('show_title')).otherwise(F.col('season_title'))
tvDF = tvDF.withColumn('season_title', season_title_isnull)
tvDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [ ]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.
STDF = tvDF.where("show_title == 'Stranger Things' AND country_name == 'United Kingdom'")
STDF.show()

### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.
STWeeks = STDF.groupBy('show_title').agg(F.count_distinct('week'))
STWeeks.show()

+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|  country_name|country_iso2|               week|category|weekly_rank|     show_title|     season_title|cumulative_weeks_in_top_10|
+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|United Kingdom|          GB|2022-08-21 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        13|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        12|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|         10|Stranger Things|Stranger Things 3|                        12|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          2|Stranger Things|Stranger Things 4|                        11|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          9|Strang

# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [ ]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25
uk_tvDF = tvDF.where("country_name == 'United Kingdom'")
Top25 = uk_tvDF.groupBy('season_title')\
            .agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks'))\
            .orderBy('max_weeks', ascending=False).limit(25).show(25)

+--------------------+---------+
|        season_title|max_weeks|
+--------------------+---------+
|   Stranger Things 4|       13|
|     Ozark: Season 4|       12|
|   Stranger Things 3|       12|
|   Stranger Things 2|       11|
|Squid Game: Season 1|       10|
|     Stranger Things|       10|
|Better Call Saul:...|       10|
|Bridgerton: Season 2|        9|
|Maid: Limited Series|        9|
|PAW Patrol: Season 6|        9|
|Inventing Anna: L...|        8|
| Money Heist: Part 5|        7|
|  Sex/Life: Season 1|        7|
|PIECES OF HER: Se...|        7|
|Clickbait: Limite...|        6|
|Sex Education: Se...|        6|
|Virgin River: Sea...|        6|
|Young Sheldon: : ...|        6|
|Young Sheldon: : ...|        6|
|The Lincoln Lawye...|        6|
|       You: Season 3|        6|
|PAW Patrol: Season 4|        6|
|The Umbrella Acad...|        5|
|Love Is Blind: Se...|        5|
|The Witcher: Seas...|        5|
+--------------------+---------+



# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [ ]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10
countriesDF = tvDF.where("show_title == 'Young Sheldon'")\
              .groupBy('season_title', 'country_name')\
              .agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks'))\
              .orderBy('max_weeks', ascending=False)

from pyspark.sql.window import Window

windowSpec = Window.partitionBy('season_title').orderBy(F.col('max_weeks').desc())

top_countries = countriesDF.withColumn('rank', F.rank().over(windowSpec))\
    .filter(F.col('rank') == 1)\
    .select('season_title', 'country_name', 'max_weeks')

result = top_countries.groupBy('season_title')\
    .agg(F.collect_list('country_name').alias('countries'))

result = result.withColumn('country', F.concat('countries')).drop('countries')

result.show(truncate=False)

+-------------------------+---------------------------------+
|season_title             |country                          |
+-------------------------+---------------------------------+
|Young Sheldon: : Season 1|[Canada, United Kingdom, Ireland]|
|Young Sheldon: : Season 2|[United Kingdom]                 |
|Young Sheldon: : Season 3|[United Kingdom]                 |
|Young Sheldon: Season 4  |[New Zealand, Australia]         |
+-------------------------+---------------------------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [ ]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.
films_maxweek_DF = filmsDF.groupBy('country_name', 'show_title').agg(F.max('cumulative_weeks_in_top_10').alias('max_weeks')).orderBy('max_weeks', ascending=False)

from pyspark.sql.window import Window

windowSpec = Window.partitionBy('country_name').orderBy(F.col('max_weeks').desc())

top_films_per_country = (
    films_maxweek_DF
    .withColumn('row_number', F.row_number().over(windowSpec))
    .filter(F.col('row_number') == 1)
    .orderBy('country_name')
    .select('country_name', 'show_title')
)

top_films_per_country.show()

+------------------+--------------------+
|      country_name|          show_title|
+------------------+--------------------+
|         Argentina|  Sonic the Hedgehog|
|         Australia| Back to the Outback|
|           Austria|Harry Potter and ...|
|           Bahamas|  A Madea Homecoming|
|           Bahrain|          Red Notice|
|        Bangladesh|        Sooryavanshi|
|           Belgium|    The Adam Project|
|           Bolivia|               Shrek|
|            Brazil|Hotel Transylvani...|
|          Bulgaria|          Red Notice|
|            Canada|How the Grinch St...|
|             Chile|  Sonic the Hedgehog|
|          Colombia|          Red Notice|
|        Costa Rica|  Sonic the Hedgehog|
|           Croatia|          Red Notice|
|            Cyprus|          Red Notice|
|    Czech Republic|The Secret Life o...|
|           Denmark|          Red Notice|
|Dominican Republic|          Red Notice|
|           Ecuador|          Red Notice|
+------------------+--------------

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [ ]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"
# films_no1 = filmsDF.select('country_name', 'week', 'show_title').where("weekly_rank == 1")
# films_no1.groupBy('country_name', 'show_title').agg(F.countDistinct('week').alias('weeks_at_1')).orderBy('country_name').show()

films_no1 = filmsDF.select('country_name', 'week', 'weekly_rank', 'show_title').where("weekly_rank == 1")
films_no1.groupBy('country_name', 'show_title')\
          .agg(F.count(F.when(F.col('weekly_rank') == 1, 1).otherwise(0)).alias('weeks_at_1'))\
          .orderBy('country_name').show()


+------------+--------------------+----------+
|country_name|          show_title|weeks_at_1|
+------------+--------------------+----------+
|   Argentina|        The Gray Man|         1|
|   Argentina|           The Vault|         1|
|   Argentina|     Army of Thieves|         2|
|   Argentina|       Blood Red Sky|         2|
|   Argentina|          Sweet Girl|         1|
|   Argentina|    The Adam Project|         2|
|   Argentina|   Through My Window|         1|
|   Argentina|      Look Both Ways|         1|
|   Argentina|       Penguin Bloom|         1|
|   Argentina|          The Guilty|         2|
|   Argentina|Bank Robbers: The...|         1|
|   Argentina|The Man from Toronto|         2|
|   Argentina|          Grand Isle|         1|
|   Argentina|         Interceptor|         1|
|   Argentina|              Hustle|         1|
|   Argentina|             Bruised|         1|
|   Argentina|              Fatale|         2|
|   Argentina|       Purple Hearts|         1|
|   Argentina

In [ ]:
# CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.
result = filmsDF.withColumn('weeks1_col', weeks1_col).groupBy('show_title', 'country_name').agg(F.sum('weeks1_col').alias('total_weeks_at_1'))
result.show(truncate=False)

+------------+---------------------------+----------------+
|country_name|show_title                 |total_weeks_at_1|
+------------+---------------------------+----------------+
|Australia   |Bloodshot                  |1               |
|Australia   |Traffik                    |0               |
|Australia   |Godzilla vs. Kong          |0               |
|Bahamas     |The Healer                 |0               |
|Bahamas     |The Hurt Locker            |0               |
|Bahrain     |Spiderhead                 |0               |
|Bahrain     |Against The Ice            |0               |
|Bahrain     |The Privilege              |0               |
|Belgium     |Heart Parade               |0               |
|Bolivia     |Men in Black: International|0               |
|Bolivia     |Retfærdighedens ryttere    |0               |
|Bolivia     |World War Z                |0               |
|Bulgaria    |G.I. Joe: The Rise of Cobra|0               |
|Canada      |Indecent Proposal         

In [ ]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.
windowSpec = Window.partitionBy("country_name").orderBy(F.desc("total_weeks_at_1"))

top_performing = (
    result
    .withColumn("rank", F.rank().over(windowSpec))
    .filter(F.col("rank") == 1)
    .select("country_name", "show_title", "total_weeks_at_1")
)

top_performing.show()


+------------+--------------------+----------------+
|country_name|          show_title|total_weeks_at_1|
+------------+--------------------+----------------+
|   Argentina|       Don't Look Up|               6|
|   Australia|    The Adam Project|               6|
|     Austria| The Tinder Swindler|              10|
|     Bahamas|              Fatale|               3|
|     Bahamas|             Bruised|               3|
|     Bahamas|        The Gray Man|               3|
|     Bahamas|           Day Shift|               3|
|     Bahamas|  A Madea Homecoming|               3|
|     Bahamas|         Senior Year|               3|
|     Bahamas|      Resort to Love|               3|
|     Bahamas|       The Sea Beast|               3|
|     Bahamas|          Sweet Girl|               3|
|     Bahamas|          Red Notice|               3|
|     Bahamas|The Man from Toronto|               3|
|     Bahamas|              Hustle|               3|
|     Bahrain|          The Guilty|           